In [1]:
import pandas as pd
import openpyxl
import kagglehub

gm = pd.read_csv("gog_games_dataset.csv")

c:\Users\79222\OneDrive\Документы\MyPythonProjects\pandas\pvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Задание 1: Разведка ниш
Игры редко принадлежат к одному чистому жанру. Обычно это смесь (например, 'RPG, Action'). Маркетологи хотят знать, какие именно комбинации жанров встречаются в магазине чаще всего.

In [2]:
genres_top = gm["genres"].value_counts().head(10)
genres_top


genres
['Action', 'Adventure', 'Platformer']         287
['Strategy', 'Real-time', 'Historical']       235
['Role-playing', 'Turn-based', 'Fantasy']     212
['Role-playing', 'Adventure', 'Fantasy']      206
['Action', 'Adventure', 'Fantasy']            189
['Adventure', 'Point-and-click', 'Puzzle']    187
['Strategy', 'Turn-based', 'Sci-fi']          173
['Adventure', 'Visual Novel', 'Narrative']    167
['Strategy', 'Real-time', 'Sci-fi']           150
['Action', 'Role-playing', 'Fantasy']         134
Name: count, dtype: int64

Задание 2: Ценовые сегменты
Руководство хочет понять, много ли у нас "премиальных" игр. Отфильтруйте и покажите игры, которые стоят дороже 50 евро. Сколько таких игр нашлось?

In [36]:
super_game = gm[gm["amount"] > 50]
super_game[['title', 'amount']]

,title,amount
45,Kingdom Come: Deliverance II,50.99
71,Europa Universalis IV: Prestige Collection,71.99
83,Chernobylite Premium Edition,54.99
97,Neptunia Virtual Stars - VIP Edition,61.19
100,Death Howl,91.19
...,...,...
10666,Combat Mission Battle for Normandy Complete,110.39
10814,Mars Tactics,91.19
10821,Capital Command,91.19
10862,MechWarrior 5: Mercenaries - JumpShip Edition,59.99


Задание 3: "Подарочный сертификат"
У клиента есть подарочный сертификат ровно на 15 евро. Он хочет купить одну крутую игру и потратить сертификат максимально эффективно (чтобы не осталось мелкой сдачи).

Найдите игры, которые попадают в узкий ценовой коридор:

Цена (amount) от 12 до 15 евро (включительно).
Рейтинг (rating) строго выше 40.
Выведите название, цену и рейтинг. Отсортируйте результат по рейтингу (лучшие — сверху).

In [12]:
price = gm['amount'].between(12, 15)
rat = gm['rating'] > 40
prize_card = gm[price & rat]
sort_prize_card = prize_card.sort_values(by=['rating', 'amount'], ascending=[False, False])
sort_prize_card[['title', 'amount', 'rating']]

,title,amount,rating
2245,Star of Providence,14.79,49
4700,Stardew Valley,13.69,48
1075,Desperados III Digital Deluxe Edition,12.56,48
2944,Hollow Knight,14.79,47
5443,Dishonored: Complete Collection,14.01,47
...,...,...,...
4241,D4: Dark Dreams Don't Die - Season One,13.69,41
4793,Stories: The Path of Destinies,13.69,41
6563,COLLAPSED,12.99,41
3685,Littlewood,12.49,41


Задание 4: Работа с конкурентами
Мы хотим посмотреть, как обстоят дела у конкретного разработчика. Пусть это будет 'Bethesda Softworks'. Создайте маску (фильтр), чтобы найти все игры этого разработчика. Отсортируйте их по рейтингу (от лучшего к худшему).

In [19]:
mask = gm['developer'] == 'Bethesda Softworks'
dev = gm.loc[mask]
dev_sort = dev.sort_values(by='rating', ascending=False)
dev_sort[['id', 'developer', 'rating']]

,id,developer,rating
4404,1440163901,Bethesda Softworks,48
4402,1440163341,Bethesda Softworks,36
4401,1440162836,Bethesda Softworks,35
4302,1435828982,Bethesda Softworks,0
4303,1435829353,Bethesda Softworks,0


Задание 5: Математика скидок
Босс одобрил скидку 20% на ВСЕ игры в датасете.

Создайте новую колонку discount_price.
В ней должна быть цена (amount), умноженная на 0.8.
Выведите таблицу с колонками title, amount, discount_price и убедитесь, что цена снизилась.

In [22]:
gm['discount_price'] = pd.Series(gm['amount'] * 0.8)
gm[['title', 'amount', 'discount_price']]


,title,amount,discount_price
0,Descent: Freespace Battle Pack,9.99,7.992
1,Freespace 2,9.99,7.992
2,MDK,7.99,6.392
3,Messiah,11.99,9.592
4,Lionheart: Legacy of the Crusader,7.99,6.392
...,...,...,...
10891,Age of Wonders: Planetfall - Premium Edition,16.29,13.032
10892,Knights of the Chalice Demo,0.00,0.000
10893,Combat Mission Cold War,30.49,24.392
10894,Sudden Strike 4,5.99,4.792


Задание 6: Самая переоцененная игра
Найдите игру, у которой самое плохое соотношение цены и качества.

Создайте метрику "Жадность" (greed_score): Цена деленная на Рейтинг (amount / rating). Чем выше цена и ниже рейтинг, тем выше "жадность".
Отсортируйте таблицу по этому показателю по убыванию.
Выведите топ-3 игры с самым высоким показателем "жадности".

In [34]:
price = gm.loc[gm['amount'] > 0]
gm_l = price.loc[price['rating'] > 0].copy()
gm_l['greed_score'] = gm_l['amount'] / gm_l['rating']
gm_sort = gm_l.sort_values(by='greed_score', ascending=False)
gm_sort[['title', 'amount', 'rating', 'greed_score']].head(3)

,title,amount,rating,greed_score
10425,Talisman: Digital Classic Edition - DLC Bundle,27.52,10,2.7520
5269,Mount & Blade II: Bannerlord - War Sails,91.19,44,2.0725
4581,No Man's Sky,54.99,30,1.8330


Задание 7: Аналитический отчет для инвестора
Инвестор просит составить "Топ-3 перспективных жанра" для вложения денег. Критерии "перспективности" он не дал, придумайте сами.

Задача: Выберите метрику, отфильтруйте "мусор" и выдайте 3 жанра.

In [52]:
from collections import defaultdict
gm_rated = gm[gm['overallAvgRating'] > 0]
gm_rated = gm_rated[gm_rated['reviewCount'] > 30]
comb_genre_count = gm_rated["genres"].value_counts()
genre_individual = defaultdict(int)
for genre_str, count in comb_genre_count.items():
    # Преобразуем строку жанров в список
    # Пример: "['Action', 'Adventure']" → ['Action', 'Adventure']
    genres = eval(genre_str)
    for genre in genres:
        genre_individual[genre] += count

genre_individual = dict(genre_individual)
genre_individual
genre_df = pd.DataFrame(list(genre_individual.items()), columns=['genre', 'count'])
genre_top = genre_df.sort_values(by="count", ascending=False)
genre_top.head(5)

,genre,count
1,Adventure,697
9,Action,695
0,Role-playing,432
2,Fantasy,425
6,Strategy,423
